In [2]:
!pip install -q catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.9 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = '/content/drive/MyDrive/multiclass/'

Mounted at /content/drive


In [15]:
org = pd.read_csv(f'{DRIVE_PATH}/cirrhosis.csv')
org.dropna(axis=0, how='any', inplace=True)
org.rename(columns={'ID': 'id'}, inplace=True)
df = pd.read_csv(f'{DRIVE_PATH}/train.csv')
df = pd.concat([df,org], axis=0)

df['Age'] = df['Age']/365
df['Age'] = np.ceil(df['Age'])


X_  = df.drop(['id','Status'],axis=1)
y_  = df['Status']


X_new = X_.replace(True,1).replace(False,0).replace('M',1).replace('F',0).replace('Y',1).replace('N',0).replace('S',1).replace('Placebo',1).replace('D-penicillamine',0)
y_new = LabelEncoder().fit_transform(y_)
Params = {'max_depth': 6,
          'learning_rate': 0.03711180731953549,
          'n_estimators': 390,
          'min_child_weight': 8,
          'gamma': 8.878825928653752e-05,
          'subsample': 0.5871365948757953,
          'colsample_bytree': 0.175999259700574,
          'reg_alpha': 1.0782222832975881e-08,
          'reg_lambda': 0.002131207411566484}
clf = XGBClassifier(**Params)
sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber")
p6={'n_iter': 2500,
    'verbose': -1,
    'objective': 'cross_entropy',
    'metric': 'auc',
    'learning_rate': 0.01,
    'colsample_bytree': 0.78,
    'colsample_bynode': 0.8,
    'lambda_l1': 4.562963348932286,
    'lambda_l2': 2.97485,
    'min_data_in_leaf': 115,
    'max_depth': 23,
    'max_bin': 898}

lgb=LGBMClassifier(**p6)
cat=CatBoostClassifier(iterations=2000,
                        verbose=0,
                        l2_leaf_reg=6.6591278779517808,
                        learning_rate=0.1,
                        subsample = 0.4,
                        allow_const_label=True,loss_function = 'CrossEntropy')
weights = [0.6,0.4]


ensemble = VotingClassifier(estimators=[('xgb',clf),
                                        ('sgd', sgd_model)

                                        #('lgb',lgb),
                                        #('cat', cat)
                                        ],
                            weights=weights, voting='soft', n_jobs=-1)

In [16]:
ensemble.fit(X_new, y_new)

VotingClassifier(estimators=[('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.175999259700574,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            gamma=8.878825928653752e-05,
                                            grow_policy=None,
                                            importance_type=None,
                                            interactio...
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=6,
                                            max_leaves=None, min_child_weight=8,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=390, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=8000, tol=0.0001))],
                 n_jobs=-1, voting='soft', weights=[0.6, 0.4])

In [17]:
df_test = pd.read_csv(f'{DRIVE_PATH}/test.csv')
df_test['Age'] = df_test['Age']/365
df_test['Age'] = np.ceil(df_test['Age'])


In [19]:

sub = ensemble.predict_proba(df_test.drop('id',axis=1).replace(True,1).replace(False,0).replace('M',1).replace('F',0).replace('Y',1).replace('N',0).replace('S',1).replace('Placebo',1).replace('D-penicillamine',0))
submission = pd.DataFrame(data=sub,
                          columns =['Status_C', 'Status_CL', 'Status_d'],
                          index=df_test['id'])
submission.reset_index().to_csv(f'{DRIVE_PATH}/sub_ensemble.csv',index=False)

